In [259]:
import pandas as pd

#需要安装额外库：pip install chardet
import chardet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [260]:
#导入数据
medal = pd.read_csv(r'D:\VScode\AllCode\美赛\2025\code\clean_data\medal_NOC_clean.csv')
ath = pd.read_csv(r'D:\VScode\AllCode\美赛\2025\code\clean_data\athlete_new_data.csv')
pro = pd.read_csv(r'D:\VScode\AllCode\美赛\2025\code\clean_data\pro_clean.csv')
medal = medal.drop(columns=["Unnamed: 0"])
pro = pro.drop(columns=["Unnamed: 0"])

#删除1906年的运动员数据
ath = ath[ath["Year"] != "1906"]
#删除冰雪运动的运动员
ath = ath[~ath["Sport"].isin(["Skating","Ice Hockey"])]
ath = ath.rename(columns ={"NOC":"RNOC"})
#过程中发现ath中的art alpinism aeronautics项目已经不存在了，并且没有在pro数据集中记录
#所以删除ath中有关数据
ath = ath[~ath["Sport"].isin(["Art Competitions","Alpinism","Aeronautics"])]

In [261]:
medal.head()

,Rank,NOC,Gold,Silver,Bronze,Total,Year,RNOC
0,1,United States,11,7,2,20,1896,USA
1,2,Greece,10,18,19,47,1896,GRE
2,3,Germany,6,5,2,13,1896,SAA
3,4,France,5,4,2,11,1896,FRA
4,5,Great Britain,2,3,2,7,1896,GBR


In [262]:
ath.head()

,Name,Sex,RNOC,Year,Sport,Event,Medal,Country
0,A Dijiang,M,CHN,1992,Basketball,Basketball Men's Basketball,No medal,China
1,A Lamusi,M,CHN,2012,Judo,Judo Men's Extra-Lightweight,No medal,China
2,Gunnar Aaby,M,DEN,1920,Football,Football Men's Football,No medal,Denmark
3,Edgar Aabye,M,DEN,1900,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark
4,Cornelia (-strannood),F,NED,1932,Athletics,Athletics Women's 100 metres,No medal,Netherlands


In [263]:
pro.head()

,Sport,Code,1896,1900,1904,1908,1912,1920,1924,1928,...,1988,1992,1996,2000,2004,2008,2012,2016,2020,2024
0,Aquatics,SWA,0,0,0,0,0,0,0,0,...,2.0,2,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,Aquatics,DIV,0,0,2,2,4,5,5,4,...,4.0,4,4.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
2,Aquatics,OWS,0,0,0,0,0,0,0,0,...,0.0,0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0
3,Aquatics,SWM,4,7,9,6,9,10,11,11,...,31.0,31,32.0,32.0,32.0,32.0,32.0,32.0,35.0,35.0
4,Aquatics,WPO,0,1,1,1,1,1,1,1,...,1.0,1,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [264]:
#特征构造：参赛国角度
#国家参加的项目数量占比
pro_year = pro.drop(columns = ["Sport","Code"])
TSport_dict = pro_year.iloc[71].to_dict()
Tevents_dict = pro_year.iloc[69].to_dict()
# 转换 TSport_dict 的键为 int64
TSport_dict = {np.int64(k): v for k, v in TSport_dict.items()}

# 转换 Tevents_dict 的键为 int64
Tevents_dict = {np.int64(k): v for k, v in Tevents_dict.items()}
print(TSport_dict)
print(Tevents_dict)
medal["TSport"] = medal["Year"].map(TSport_dict)
medal["Tevent"] = medal["Year"].map(Tevents_dict)

{np.int64(1896): '11', np.int64(1900): 20, np.int64(1904): 16, np.int64(1908): 22, np.int64(1912): 14, np.int64(1920): 22, np.int64(1924): 17, np.int64(1928): 14, np.int64(1932): 14, np.int64(1936): 19, np.int64(1948): 17, np.int64(1952): 17.0, np.int64(1956): 17, np.int64(1960): 17.0, np.int64(1964): 19, np.int64(1968): 18, np.int64(1972): 21, np.int64(1976): 21.0, np.int64(1980): 21.0, np.int64(1984): 21, np.int64(1988): 23.0, np.int64(1992): 25, np.int64(1996): 25.0, np.int64(2000): 27.0, np.int64(2004): 27.0, np.int64(2008): 27.0, np.int64(2012): 26.0, np.int64(2016): 28.0, np.int64(2020): 33.0, np.int64(2024): 32.0}
{np.int64(1896): '43', np.int64(1900): 97, np.int64(1904): 95, np.int64(1908): 110, np.int64(1912): 102, np.int64(1920): 156, np.int64(1924): 126, np.int64(1928): 109, np.int64(1932): 117, np.int64(1936): 129, np.int64(1948): 136, np.int64(1952): 149.0, np.int64(1956): 151, np.int64(1960): 150.0, np.int64(1964): 163, np.int64(1968): 172, np.int64(1972): 195, np.int64(1

In [265]:
# Calculate the number of participants per country per year
participants_per_country_year = ath.groupby(['Year', 'RNOC']).size().reset_index(name='Participants')
participants_per_country_year = participants_per_country_year.rename(columns = {"NOC":"RNOC"})
print(participants_per_country_year)

#计算每个国家每年参加的男性占比

# Calculate the number of awards per athlete
awards_per_athlete = ath.dropna(subset=['Medal']).groupby('Name')['Medal'].count().reset_index(name='Awards')
#print(awards_per_athlete)

# Calculate the number of participations per athlete
participations_per_athlete = ath.groupby('Name').size().reset_index(name='Participations')
#print(participations_per_athlete)

# Calculate the number of each type of medal per athlete
medals_per_athlete = ath.dropna(subset=['Medal']).groupby(['Name', 'Medal']).size().unstack(fill_value=0).reset_index()
#print(medals_per_athlete)

# Merge awards, participations, and medals into a single DataFrame
athlete_summary = participations_per_athlete.merge(awards_per_athlete, on='Name', how='left').merge(medals_per_athlete, on='Name', how='left').fillna(0)
print(athlete_summary)

      Year RNOC  Participants
0     1896  AUS             5
1     1896  AUT             8
2     1896  DEN            15
3     1896  FRA            26
4     1896  GBR            25
...    ...  ...           ...
3203  2024  VIE            20
3204  2024  VIN             4
3205  2024  YEM             4
3206  2024  ZAM            32
3207  2024  ZIM             7

[3208 rows x 3 columns]
                 Name  Participations  Awards  Bronze  Gold  No medal  Silver
0        (jr) Larocca               1       1       0     0         1       0
1       . Chadalavada               1       1       0     0         1       0
2              . Deni               1       1       0     0         1       0
3                 671               1       1       1     0         0       0
4            A Alayed               1       1       0     0         1       0
...               ...             ...     ...     ...   ...       ...     ...
128156    zge Emberci               1       1       0     0         1

In [266]:
#加入特征：参与的运动员数量
medal = pd.merge(participants_per_country_year,medal,on = ["RNOC","Year"])

In [267]:
#添加一列，运动员是否获得了奖牌
ath["isMedal"] = (ath['Medal'] != 'No medal').astype(int)

In [268]:
#加入特征：男运动员总数
ath_new = ath.copy()
ath_new["Sex"] = ath_new["Sex"].map({"M" :1,"F":0})
ath_new["MSum"] = ath_new.groupby(["RNOC","Year"])["Sex"].transform("sum")

# 创建映射字典：(key1, key2) -> cumulative_sum
# 使用 `.drop_duplicates()` 只保留每组的第一个累加值
msum_noc_year_dict = ath_new.drop_duplicates(['RNOC', 'Year']).set_index(['RNOC', 'Year'])['MSum'].to_dict()


medal['man_sum'] = medal.apply(lambda row: msum_noc_year_dict.get((row['RNOC'], row['Year']), 0), axis=1)


In [269]:
#我们需要判断哪些项目为团体项目
#https://www.kaggle.com/code/chadalee/olympics-data-cleaning-exploration-prediction
#对同国家同event同Year同medal的运动员数进行累加，如果大于1，则为团体项目
identify_team_events = pd.pivot_table(ath,
                                      index = ['RNOC', 'Year', 'Event'],
                                      columns = 'Medal',
                                      values = 'isMedal',
                                      aggfunc = 'sum',
                                     fill_value = 0).reset_index()

identify_team_events = identify_team_events.loc[identify_team_events['Gold'] > 1, :]

team_sports = identify_team_events['Event'].unique()
#其中部分项目并非团体，只是单人项目出现了并列第一
remove_sports = ["Gymnastics Women's Balance Beam", 
                 "Gymnastics Men's Horizontal Bar", 
                 "Gymnastics Men's Pommelled Horse",
                 "Swimming Women's 100 metres Freestyle",
                   "Swimming Men's 50 metres Freestyle",
                   "Athletics Men's Pole Vault",
                   "Men's Park",
                   "Women's Singles",
                   "Gymnastics Men's Horse Vault",

                   ]
team_sports = list(set(team_sports) - set(remove_sports))
# if an event name matches with one in team sports, then it is a team event. Others are singles events.
ath["isTeam"] =ath['Event'].map(lambda x: x in team_sports)

In [270]:
#加入特征：参加的团体运动数量
ath_team = ath[ath["isTeam"] == True]
ath_team = ath_team[["Event","RNOC","Year"]].drop_duplicates()

team_event_counts = ath_team.groupby(["RNOC","Year"]).size().reset_index(name = "team_event_count")
medal = pd.merge(medal,team_event_counts,on = ["RNOC","Year"])

In [271]:
#手动进行映射
athSport_to_proSport = {
    # 1
    'Basketball': 'Basketball',
    'Judo': 'Judo',
    'Football': 'Football',
    'Tug-Of-War': 'Tug of War',  
    'Athletics': 'Athletics',
    'Swimming': 'Aquatics',  
    
    # 2
    'Badminton': 'Badminton',
    'Sailing': 'Sailing',
    'Gymnastics': 'Gymnastics',
    #'Art Competitions': 'Others',  # 不在大类列表中，可做特殊处理
    'Handball': 'Handball',
    'Weightlifting': 'Weightlifting',
    'Wrestling': 'Wrestling',
    'Water Polo': 'Aquatics',      # 水球也是属于大类“Aquatics”
    
    # 3
    'Hockey': 'Field hockey',      # 可能需要根据具体语境确认
    'Rowing': 'Rowing',
    'Fencing': 'Fencing',
    'Equestrianism': 'Equestrian', # 贴近大类名称
    'Shooting': 'Shooting',
    'Boxing': 'Boxing',
    'Taekwondo': 'Taekwondo',
    'Cycling': 'Cycling',
    
    # 4
    'Diving': 'Aquatics',          # 跳水也属于“Aquatics”
    'Canoeing': 'Canoeing',
    'Tennis': 'Tennis',
    'Modern Pentathlon': 'Modern Pentathlon',
    'Golf': 'Golf',
    'Softball': 'Baseball and Softball',
    'Archery': 'Archery',
    'Volleyball': 'Volleyball',
    
    # 5
    'Synchronized Swimming': 'Aquatics',  # 现多称“Artistic Swimming”，同属“Aquatics”
    'Table Tennis': 'Table Tennis',
    'Baseball': 'Baseball and Softball',
    'Rhythmic Gymnastics': 'Gymnastics',
    'Rugby Sevens': 'Rugby',
    'Trampolining': 'Gymnastics',         # 奥运归类于体操
    'Beach Volleyball': 'Volleyball',
    'Triathlon': 'Triathlon',
    
    # 6
    'Rugby': 'Rugby',
    'Lacrosse': 'Lacrosse',
    'Polo': 'Polo',
    'Cricket': 'Cricket',
    'Racquets': 'Rackets',         # 大类中拼写为 Rackets
    'Motorboating': 'Water Motorsports',
    'Croquet': 'Croquet',
    'Jeu De Paume': 'Jeu de Paume',
    
    # 7
    'Roque': 'Roque',
    'Basque Pelota': 'Basque Pelota',
    #'Alpinism': 'Others',          # 不在大类列表中，可做特殊处理
    #'Aeronautics': 'Others',       # 不在大类列表中
    'Cycling Road': 'Cycling',
    'Artistic Gymnastics': 'Gymnastics',
    'Karate': 'Karate',
    'Baseball/Softball': 'Baseball and Softball',
    
    # 8
    'Trampoline Gymnastics': 'Gymnastics',
    'Marathon Swimming': 'Aquatics',
    'Canoe Slalom': 'Canoeing',
    'Surfing': 'Surfing',
    'Canoe Sprint': 'Canoeing',
    'Cycling BMX Racing': 'Cycling',
    'Equestrian': 'Equestrian',    # 同“Equestrianism”
    'Artistic Swimming': 'Aquatics',
    
    # 9
    'Cycling Track': 'Cycling',
    'Skateboarding': 'Skateboarding',
    'Cycling Mountain Bike': 'Cycling',
    '3x3 Basketball': 'Basketball',
    'Cycling BMX Freestyle': 'Cycling',
    'Sport Climbing': 'Sport Climbing',
    'Marathon Swimming, Swimming': 'Aquatics', 
    'Breaking': 'Breaking',
    
    # 10 （多项目合并的例子，都可视作“Cycling”或另外拆分）
    'Cycling Road, Cycling Track': 'Cycling',
    'Cycling Road, Cycling Mountain Bike': 'Cycling',
    'Cycling Road, Triathlon': 'Cycling',   
    '3x3 Basketball, Basketball': 'Basketball'
}
ath["type_sport"] = ath["Sport"].map(athSport_to_proSport)

In [272]:
#统计各类型项目（Sport）的参加人数
#算法1：团体项目只能算一个人
#算法2：团体项目所有人都算

# 1. 划分团队项目和个人项目
ath_team = ath[ath['isTeam'] == 1].copy()    # 团体项目
ath_individual = ath[ath['isTeam'] == 0].copy()  # 个人项目

# 2. 对“团队项目”数据进行去重：
# 同一个 (country, year, type_sport) 只保留一条
# 因为同一队的多名运动员只算 1
ath_team_unique = ath_team.drop_duplicates(subset=['RNOC', 'Year', 'type_sport']).copy()
# 此时每支队伍只剩下 1 条记录，可以给一个计数列记为 1
ath_team_unique['count'] = 1
#print(ath_team_unique)

# 3. 对“个人项目”数据，每个运动员都算 1
ath_individual['count'] = 1
#print(ath_individual)

# 4. 合并
df_combined = pd.concat([ath_team_unique, ath_individual], ignore_index=True)

# 5. 按 (country, year, type_sport) 分组汇总 count
type_sum_uq = df_combined.groupby(['RNOC', 'Year', 'type_sport'], as_index=False)['count'].sum()

# 6. result 即为所需结果
print(type_sum_uq)

ath_all = ath.copy()
ath_all["count"] = 1
type_sum_all = ath_all.groupby(['RNOC', 'Year', 'type_sport'], as_index=False)['count'].sum()
print(type_sum_all)

      RNOC  Year    type_sport  count
0      AFG  1936     Athletics      3
1      AFG  1936  Field hockey      1
2      AFG  1948  Field hockey      1
3      AFG  1948      Football      1
4      AFG  1956  Field hockey      1
...    ...   ...           ...    ...
24131  ZIM  2020          Golf      1
24132  ZIM  2020        Rowing      1
24133  ZIM  2024      Aquatics      2
24134  ZIM  2024     Athletics      4
24135  ZIM  2024        Rowing      1

[24136 rows x 4 columns]
      RNOC  Year    type_sport  count
0      AFG  1936     Athletics      3
1      AFG  1936  Field hockey     13
2      AFG  1948  Field hockey     14
3      AFG  1948      Football     11
4      AFG  1956  Field hockey     12
...    ...   ...           ...    ...
24131  ZIM  2020          Golf      1
24132  ZIM  2020        Rowing      1
24133  ZIM  2024      Aquatics      2
24134  ZIM  2024     Athletics      4
24135  ZIM  2024        Rowing      1

[24136 rows x 4 columns]


In [273]:
#加入特征：各sport参加人数（uq和all两种）
# 1. 生成宽表：每个 type_sport 变成单独的一列
uq_pivot = type_sum_uq.pivot_table(
    index=['RNOC', 'Year'], 
    columns='type_sport', 
    values='count', 
    aggfunc='sum',       # 通常是求和，也可写成 np.sum
    fill_value=0         # 将缺失值填充为 0
)

all_pivot = type_sum_all.pivot_table(
    index=['RNOC', 'Year'], 
    columns='type_sport', 
    values='count', 
    aggfunc='sum',       # 通常是求和，也可写成 np.sum
    fill_value=0         # 将缺失值填充为 0
)


# 2. 恢复索引
uq_pivot = uq_pivot.reset_index()
all_pivot = all_pivot.reset_index()

#暂时先用总人数吧
medal = pd.merge(medal,all_pivot,on = ["RNOC","Year"])

In [274]:
medal.head()

,Year,RNOC,Participants,Rank,NOC,Gold,Silver,Bronze,Total,TSport,...,Surfing,Table Tennis,Taekwondo,Tennis,Triathlon,Tug of War,Volleyball,Water Motorsports,Weightlifting,Wrestling
0,1896,AUS,5,8,Australia,2,0,0,2,11,...,0,0,0,2,0,0,0,0,0,0
1,1896,GBR,25,5,Great Britain,2,3,2,7,11,...,0,0,0,4,0,0,0,0,2,1
2,1896,GRE,148,2,Greece,10,18,19,47,11,...,0,0,0,13,0,0,0,0,4,2
3,1900,AUS,6,9,Australia,2,0,3,5,20,...,0,0,0,0,0,0,0,0,0,0
4,1900,BEL,100,5,Belgium,6,7,4,17,20,...,0,0,0,0,0,0,0,0,0,0


至此，运动员特征已经加入：
参加的总人数+男性人数（男性占比）
参加团体项目的数量
各项目(sport)参赛总人数




In [275]:
medal["menPropotion"] = medal["man_sum"] / medal["Participants"]
#medal = medal.drop(columns = ["man_sum"])

In [277]:
medal.head()

,Year,RNOC,Participants,Rank,NOC,Gold,Silver,Bronze,Total,TSport,...,Table Tennis,Taekwondo,Tennis,Triathlon,Tug of War,Volleyball,Water Motorsports,Weightlifting,Wrestling,menPropotion
0,1896,AUS,5,8,Australia,2,0,0,2,11,...,0,0,2,0,0,0,0,0,0,1.0
1,1896,GBR,25,5,Great Britain,2,3,2,7,11,...,0,0,4,0,0,0,0,2,1,1.0
2,1896,GRE,148,2,Greece,10,18,19,47,11,...,0,0,13,0,0,0,0,4,2,1.0
3,1900,AUS,6,9,Australia,2,0,3,5,20,...,0,0,0,0,0,0,0,0,0,1.0
4,1900,BEL,100,5,Belgium,6,7,4,17,20,...,0,0,0,0,0,0,0,0,0,1.0


In [276]:
medal.to_csv(r"D:\VScode\AllCode\美赛\2025\code\clean_data\medal_feature_ath.csv")